In [125]:
import sys
import os

# Defina o caminho base manualmente
base_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

# Adicione o diretório base ao caminho de pesquisa de módulos
sys.path.append(base_path)

# Agora você pode importar utils e outros módulos
import utils
import setups.emas as emas
import setups.stopgain as StopGain
import setups.stoploss as StopLoss

import numpy as np
import itertools
import pandas as pd
from datetime import datetime
import requests
from datetime import datetime, timedelta

In [126]:
def adjust_date(start_date):
    start_datetime = datetime.strptime(start_date, '%Y-%m-%d')
    days_to_subtract = 10.40625
    new_datetime = start_datetime - timedelta(days=days_to_subtract)
    return new_datetime.strftime('%Y-%m-%d')

def calculate_sharpe_ratio(returns, risk_free_rate=0.05):
    excess_returns = returns - risk_free_rate
    mean_excess_return = np.mean(excess_returns)
    std_excess_return = np.std(excess_returns)
    sharpe_ratio = mean_excess_return / std_excess_return if std_excess_return != 0 else 0
    return sharpe_ratio

def evaluate_performance(data, short_period, long_period, quantidade_de_candles_para_stop, ratio):
    saldo = 1000
    taxa_por_operacao = 0.0153
    comprado = False

    data['ema_short'] = data['close'].ewm(span=short_period, adjust=False).mean()
    data['ema_long'] = data['close'].ewm(span=long_period, adjust=False).mean()

    max_saldo = saldo
    min_saldo_since_max = saldo
    max_drawdown = 0
    perdas = []
    ganhos = []

    results = {}
    trades = []

    for i in range(max(short_period, long_period), len(data)):
        year = data['open_time'].iloc[i - 1].year
        month = data['open_time'].iloc[i - 1].month

        if year not in results:
            results[year] = {}
        if month not in results[year]:
            results[year][month] = {
                'open_trades': 0,
                'lucro': 0,
                'successful_trades': 0,
                'failed_trades': 0,
                'perda_percentual_total': 0,
                'saldo_inicial': saldo,
                'saldo_final': saldo,
                'max_drawdown': 0
            }

        if comprado:
            if StopLoss.sell_stoploss(data['low'].iloc[i - 1], stoploss):
                loss_percentage = utils.calculate_loss_percentage(buy_price, stoploss)
                results[year][month]['failed_trades'] += 1
                results[year][month]['perda_percentual_total'] += loss_percentage + taxa_por_operacao
                saldo -= saldo * ((loss_percentage + taxa_por_operacao) / 100)
                results[year][month]['saldo_final'] = saldo
                comprado = False

                trade['close_price'] = stoploss
                trade['close_time'] = data['open_time'].iloc[i - 1]
                trade['outcome'] = loss_percentage
                trade['result'] = 'StopLoss'
                trades.append(trade)
                perdas.append(-(loss_percentage + taxa_por_operacao))

                if saldo < min_saldo_since_max:
                    min_saldo_since_max = saldo
                drawdown = (max_saldo - min_saldo_since_max) / max_saldo * 100

                if drawdown > max_drawdown:
                    max_drawdown = drawdown

                results[year][month]['max_drawdown'] = max_drawdown

                continue

            elif StopGain.sell_stopgain(data['high'].iloc[i - 1], stopgain):
                profit = utils.calculate_gain_percentage(buy_price, stopgain)
                results[year][month]['lucro'] += profit - taxa_por_operacao
                results[year][month]['successful_trades'] += 1
                saldo += saldo * ((profit - taxa_por_operacao) / 100)
                results[year][month]['saldo_final'] = saldo
                comprado = False

                trade['close_price'] = stopgain
                trade['close_time'] = data['open_time'].iloc[i - 1]
                trade['outcome'] = profit
                trade['result'] = 'StopGain'
                trades.append(trade)

                ganhos.append(profit - taxa_por_operacao)

                if saldo > max_saldo:
                    max_saldo = saldo
                    min_saldo_since_max = saldo

                continue

        if not comprado:
            if emas.buy_double_ema_breakout(data.iloc[i - 5:i], 'ema_short', 'ema_long'):
                results[year][month]['open_trades'] += 1
                buy_price = data['high'].iloc[i - 2]
                stoploss = StopLoss.set_sell_stoploss_min_candles(data.iloc[i - quantidade_de_candles_para_stop:i], quantidade_de_candles_para_stop)
                if taxa_por_operacao != 0:
                    saldo -= saldo * taxa_por_operacao / 100
                results[year][month]['saldo_final'] = saldo
                stopgain = StopGain.set_sell_stopgain_percentage(buy_price, ratio)
                comprado = True

                trade = {
                    'open_time': data['open_time'].iloc[i - 1],
                    'buy_price': buy_price,
                    'stoploss': stoploss,
                    'stopgain': stopgain,
                    'close_price': 0,
                    'close_time': 0,
                    'outcome': 0,
                    'result': ''
                }
                continue

    performance_metric = saldo - 1000  # Exemplos de métrica de desempenho: saldo final menos saldo inicial
    return performance_metric, saldo, trades, results, max_drawdown, ganhos, perdas



In [127]:
# Carregar os dados dos arquivos CSV
df_15m = pd.read_csv('BTC_15m_candles.csv')
df_5m = pd.read_csv('BTC_5m_candles.csv')

# Padronizar os nomes das colunas para minúsculas e substituir espaços por underscores
df_15m.columns = [col.lower().replace(' ', '_') for col in df_15m.columns]
df_5m.columns = [col.lower().replace(' ', '_') for col in df_5m.columns]

# Definir o tempo inicial e o tempo final
start_date = '2023-08-01'
end_date = '2024-08-01'

adjusted_start_date = adjust_date(start_date)

# Converter strings de data para datetime
start_datetime = pd.to_datetime(start_date)
end_datetime = pd.to_datetime(end_date)

# Filtrar os dados com base no tempo inicial e final
df_15m['open_time'] = pd.to_datetime(df_15m['open_time'])
df_5m['open_time'] = pd.to_datetime(df_5m['open_time'])
df_15m = df_15m[(df_15m['open_time'] >= start_datetime) & (df_15m['open_time'] <= end_datetime)]
df_5m = df_5m[(df_5m['open_time'] >= start_datetime) & (df_5m['open_time'] <= end_datetime)]

In [128]:
# Definir os parâmetros e seus intervalos
param_grid = {
    'short_period': [9],
    'long_period': [21],
    'ratio': [3.5],
    #'ratio': np.round(np.arange(1.2, 4.3, 0.3), 1).tolist(),
    'timeframe': ['15m'],
    'quantidade_de_candles_para_stop': [14],
    #'quantidade_de_candles_para_stop': list(range(5, 21)),
    'ativo': ['BTCUSDT'],
    'setup': ['EMA']
}


In [129]:
param_combinations = list(itertools.product(*param_grid.values()))

best_performance = None
best_params = None
best_result = None

In [130]:
best_performance = None
best_params = None
best_result = None

for combination in param_combinations:
    params = dict(zip(param_grid.keys(), combination))
    
    # Selecionar o dataframe correto com base no timeframe
    if params['timeframe'] == '15m':
        data = df_15m.copy()
    else:
        data = df_5m.copy()

    # Avaliar o desempenho da estratégia com os parâmetros atuais
    performance_score, final_saldo, trades, results, max_drawdown, ganhos, perdas = evaluate_performance(
        data, params['short_period'], params['long_period'], params['quantidade_de_candles_para_stop'], params['ratio']
    )
    
    # Registrar a melhor combinação
    if best_performance is None or performance_score > best_performance:
        best_performance = performance_score
        best_params = params
        best_result = {
            'final_saldo': final_saldo,
            'trades': trades,
            'results': results,
            'max_drawdown': max_drawdown,
            'ganhos': ganhos,
            'perdas': perdas
        }

# Imprimir os resultados detalhados do melhor desempenho
if best_result:
    saldo = best_result['final_saldo']
    trades = best_result['trades']
    results = best_result['results']
    max_drawdown = best_result['max_drawdown']
    ganhos = best_result['ganhos']
    perdas = best_result['perdas']

    overall_sharpe_ratio = calculate_sharpe_ratio(np.array(ganhos + perdas), 0.15)

    for year in results:
        print(f"Ano: {year}")
        print(f"  Operações realizadas: {sum([results[year][month]['open_trades'] for month in results[year]])}")
        print(f"  Trades de sucesso: {sum([results[year][month]['successful_trades'] for month in results[year]])}")
        print(f"  Soma dos ganhos: {sum([results[year][month]['lucro'] for month in results[year]]):.2f}%")
        try:
            print(f"  Ganho médio por trade: {sum([results[year][month]['lucro'] for month in results[year]]) / sum([results[year][month]['successful_trades'] for month in results[year]]) :.2f}%")
        except ZeroDivisionError:
            print(f"  Ganho médio por trade: 0")
        print(f"  Trades em prejuízo: {sum([results[year][month]['failed_trades'] for month in results[year]])}")
        print(f"  Soma das perdas: {sum([results[year][month]['perda_percentual_total'] for month in results[year]]):.2f}%")
        
        total_loss = sum([results[year][month]['perda_percentual_total'] for month in results[year]])
        total_failed_trades = sum([results[year][month]['failed_trades'] for month in results[year]])

        if total_failed_trades != 0:
            avg_loss_per_trade = total_loss / total_failed_trades
        else:
            avg_loss_per_trade = 0

        print(f"  Perda média por trade: {avg_loss_per_trade:.2f}%")
        
        if results[year][list(results[year].keys())[0]]['saldo_inicial'] <= results[year][list(results[year].keys())[-1]]['saldo_final']:
            print(f"  Resultado final: {((results[year][list(results[year].keys())[-1]]['saldo_final'] / results[year][list(results[year].keys())[0]]['saldo_inicial']) - 1) * 100:.2f}%")
        else:
            print(f"  Resultado final: {((1 - (results[year][list(results[year].keys())[-1]]['saldo_final'] / results[year][list(results[year].keys())[0]]['saldo_inicial'])) * -1) * 100:.2f}%")
        
        print(f"  Saldo inicial: {results[year][list(results[year].keys())[0]]['saldo_inicial']:.2f}")
        print(f"  Saldo final: {results[year][list(results[year].keys())[-1]]['saldo_final']:.2f}")
        print("Detalhes:")
        for month in results[year]:
            print(f"  Mês: {month}")
            print(f"    Operações realizadas: {results[year][month]['open_trades']}")
            print(f"    Trades de sucesso: {results[year][month]['successful_trades']}")
            print(f"    Soma dos ganhos: {results[year][month]['lucro']:.2f}%")
            try:
                print(f"    Ganho médio por trade: {results[year][month]['lucro'] / results[year][month]['successful_trades']:.2f}%")
            except ZeroDivisionError:
                print(f"    Ganho médio por trade: 0")
            print(f"    Trades em prejuízo: {results[year][month]['failed_trades']}")
            print(f"    Soma das perdas: {results[year][month]['perda_percentual_total']:.2f}%")
            
            failed_trades = results[year][month]['failed_trades']

            if failed_trades != 0:
                avg_loss_per_trade = results[year][month]['perda_percentual_total'] / failed_trades
            else:
                avg_loss_per_trade = 0

            print(f"    Perda média por trade: {avg_loss_per_trade:.2f}%")
            print(f"    Drawdown máximo: {results[year][month]['max_drawdown']:.2f}%")

            if results[year][month]['saldo_inicial'] <= results[year][month]['saldo_final']:
                print(f"    Resultado final: {(results[year][month]['saldo_final'] / results[year][month]['saldo_inicial'] - 1) * 100:.2f}%")
            else:
                print(f"    Resultado final: {((1 - (results[year][month]['saldo_final'] / results[year][month]['saldo_inicial'])) * -1) * 100:.2f}%")

            print(f"    Saldo inicial: {results[year][month]['saldo_inicial']:.2f}")
            print(f"    Saldo final: {results[year][month]['saldo_final']:.2f}")
            print("-------------------")

    print("Total:")
    print(f"Operações realizadas: {sum([results[year][month]['open_trades'] for year in results for month in results[year]])}")
    print(f"Sharpe Ratio: {overall_sharpe_ratio:.2f}")
    try:
        print(f"Taxa de acerto: {sum([results[year][month]['successful_trades'] for year in results for month in results[year]]) / sum([results[year][month]['open_trades'] for year in results for month in results[year]]) * 100:.2f}%")
    except ZeroDivisionError:
        print(f"Taxa de acerto: 0")
    print(f"Trades de sucesso: {sum([results[year][month]['successful_trades'] for year in results for month in results[year]])}")
    print(f"Soma dos ganhos: {sum([results[year][month]['lucro'] for year in results for month in results[year]]):.2f}%")

    try:
        print(f"Ganho médio por trade: {sum([results[year][month]['lucro'] for year in results for month in results[year]]) / sum([results[year][month]['successful_trades'] for year in results for month in results[year]]) :.2f}%")
    except ZeroDivisionError:
        print(f"Ganho médio por trade: 0")

    print(f"Trades em prejuízo: {sum([results[year][month]['failed_trades'] for year in results for month in results[year]])}")
    print(f"Soma das perdas: {sum([results[year][month]['perda_percentual_total'] for year in results for month in results[year]]):.2f}%")
    try:
        print(f"Perda média por trade: {sum([results[year][month]['perda_percentual_total'] for year in results for month in results[year]]) / sum([results[year][month]['failed_trades'] for year in results for month in results[year]]) :.2f}%")
    except ZeroDivisionError:
        print(f"Perda média por trade: 0")
    print(f"Drawdown máximo: {max_drawdown:.2f}%")

    saldo_inicial = results[list(results.keys())[0]][list(results[list(results.keys())[0]].keys())[0]]['saldo_inicial']
    saldo_final = results[list(results.keys())[-1]][list(results[list(results.keys())[-1]].keys())[-1]]['saldo_final']

    if saldo_inicial <= saldo_final:
        print(f"Resultado final: {(saldo_final / saldo_inicial - 1) * 100:.2f}%")
    else:
        print(f"Resultado final: {((1 - (saldo_final / saldo_inicial)) * -1) * 100:.2f}%")

    print(f"Saldo inicial: {results[list(results.keys())[0]][list(results[list(results.keys())[0]].keys())[0]]['saldo_inicial']:.2f}")
    print(f"Saldo final: {results[list(results.keys())[-1]][list(results[list(results.keys())[-1]].keys())[-1]]['saldo_final']:.2f}")
    print("-------------------")
    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}: Teste finalizado: {params['ativo']} - {params['timeframe']}.")
    print(f"Setup: EMA {params['short_period']}/{params['long_period']} rompimento, stopgain ratio {params['ratio']} e stoploss {params['quantidade_de_candles_para_stop']} candles")


Ano: 2023
  Operações realizadas: 94
  Trades de sucesso: 30
  Soma dos ganhos: 104.54%
  Ganho médio por trade: 3.48%
  Trades em prejuízo: 63
  Soma das perdas: 75.50%
  Perda média por trade: 1.20%
  Resultado final: 28.52%
  Saldo inicial: 1000.00
  Saldo final: 1285.22
Detalhes:
  Mês: 8
    Operações realizadas: 29
    Trades de sucesso: 4
    Soma dos ganhos: 13.94%
    Ganho médio por trade: 3.48%
    Trades em prejuízo: 25
    Soma das perdas: 24.13%
    Perda média por trade: 0.97%
    Drawdown máximo: 12.70%
    Resultado final: -10.51%
    Saldo inicial: 1000.00
    Saldo final: 894.92
-------------------
  Mês: 9
    Operações realizadas: 12
    Trades de sucesso: 3
    Soma dos ganhos: 10.45%
    Ganho médio por trade: 3.48%
    Trades em prejuízo: 8
    Soma das perdas: 6.80%
    Perda média por trade: 0.85%
    Drawdown máximo: 14.75%
    Resultado final: 3.30%
    Saldo inicial: 894.92
    Saldo final: 924.49
-------------------
  Mês: 10
    Operações realizadas: 14
 

In [124]:
print("Melhores parâmetros encontrados:")
print(best_params)
print("Melhor desempenho obtido:")
print(best_performance)

Melhores parâmetros encontrados:
{'short_period': 9, 'long_period': 21, 'ratio': 3.5, 'timeframe': '15m', 'quantidade_de_candles_para_stop': 14, 'ativo': 'BTCUSDT', 'setup': 'EMA'}
Melhor desempenho obtido:
1510.7024993965347
